# Clean Data
This notebook intends to clean the Raw DataFrame having as outcome one or more interim DataFrame, ready to has the features engineered (next step).<br>
The Clean Data steps that might be followed in order to clean the Raw DataFrame are:<br>
1. NaN
2. Features that has same value in all rows
3. Duplicated Features(identical to other existent column)
4. High Correlation Features
5. Window Selection (0-2)

<a name="sumario"></a>
# Sumário
<!--ts-->
   * [Importe das Bibliotecas](#import)
   * [Leitura dos dados](#leitura)
   * [Dados faltantes (NaN)](#nan)
   * [Valores repetidos](#repet)
   * [Features duplicadas](#dup_feat)
   * [Nome features exame de sangue](#rename)
   * [Window Selection](#win_select)
   * [Conclusão](#conclusion)
<!--te-->

<a name="import"></a>
# Importe das Bibliotecas

## Bibliotecas Externas

In [1]:
import pandas as pd
import numpy as np

## Bibliotecas Internas

In [2]:
import sys
sys.path.insert(1, "../src/")
from clean import print_nan_count_by_feature, neighborhood_missing_data
from clean import drop_features_with_same_value_for_all_observations, plot_features_with_same_value_for_all_observations
from clean import drop_duplicated_features, plot_duplicated_features
from clean import rename_portion_of_columns, prepare_window

-----------------
Retornar ao [Sumário](#sumario)

<a name="leitura"></a>
# Leitura dos dados
Faz a leitura dos dados que serão usados para a limpeza e futura modelagem.<br>
**```df : pd.DataFrame```** é o DataFrame que receberá os valores *raw* baixados do Kaggle.

In [3]:
# Leitura do raw data desse  projeto no Github
df = pd.read_excel('https://github.com/fdrigui/covid19_icu_admission_prediction/raw/main/data/raw/Kaggle_Sirio_Libanes_ICU_Prediction.xlsx' )

# Imprime na tela todas as colunas em vez de as 10 primeiras e 10 ultimas
pd.set_option('max_columns', df.shape[1])
df.head()

,PATIENT_VISIT_IDENTIFIER,AGE_ABOVE65,AGE_PERCENTIL,GENDER,DISEASE GROUPING 1,DISEASE GROUPING 2,DISEASE GROUPING 3,DISEASE GROUPING 4,DISEASE GROUPING 5,DISEASE GROUPING 6,HTN,IMMUNOCOMPROMISED,OTHER,ALBUMIN_MEDIAN,ALBUMIN_MEAN,ALBUMIN_MIN,ALBUMIN_MAX,ALBUMIN_DIFF,BE_ARTERIAL_MEDIAN,BE_ARTERIAL_MEAN,BE_ARTERIAL_MIN,BE_ARTERIAL_MAX,BE_ARTERIAL_DIFF,BE_VENOUS_MEDIAN,BE_VENOUS_MEAN,BE_VENOUS_MIN,BE_VENOUS_MAX,BE_VENOUS_DIFF,BIC_ARTERIAL_MEDIAN,BIC_ARTERIAL_MEAN,BIC_ARTERIAL_MIN,BIC_ARTERIAL_MAX,BIC_ARTERIAL_DIFF,BIC_VENOUS_MEDIAN,BIC_VENOUS_MEAN,BIC_VENOUS_MIN,BIC_VENOUS_MAX,BIC_VENOUS_DIFF,BILLIRUBIN_MEDIAN,BILLIRUBIN_MEAN,BILLIRUBIN_MIN,BILLIRUBIN_MAX,BILLIRUBIN_DIFF,BLAST_MEDIAN,BLAST_MEAN,BLAST_MIN,BLAST_MAX,BLAST_DIFF,CALCIUM_MEDIAN,CALCIUM_MEAN,CALCIUM_MIN,CALCIUM_MAX,CALCIUM_DIFF,CREATININ_MEDIAN,CREATININ_MEAN,CREATININ_MIN,CREATININ_MAX,CREATININ_DIFF,FFA_MEDIAN,FFA_MEAN,FFA_MIN,FFA_MAX,FFA_DIFF,GGT_MEDIAN,GGT_MEAN,GGT_MIN,GGT_MAX,GGT_DIFF,GLUCOSE_MEDIAN,GLUCOSE_MEAN,GLUCOSE_MIN,GLUCOSE_MAX,GLUCOSE_DIFF,HEMATOCRITE_MEDIAN,HEMATOCRITE_MEAN,HEMATOCRITE_MIN,HEMATOCRITE_MAX,HEMATOCRITE_DIFF,HEMOGLOBIN_MEDIAN,HEMOGLOBIN_MEAN,HEMOGLOBIN_MIN,HEMOGLOBIN_MAX,HEMOGLOBIN_DIFF,INR_MEDIAN,INR_MEAN,INR_MIN,INR_MAX,INR_DIFF,LACTATE_MEDIAN,LACTATE_MEAN,LACTATE_MIN,LACTATE_MAX,LACTATE_DIFF,LEUKOCYTES_MEDIAN,LEUKOCYTES_MEAN,LEUKOCYTES_MIN,LEUKOCYTES_MAX,LEUKOCYTES_DIFF,LINFOCITOS_MEDIAN,LINFOCITOS_MEAN,LINFOCITOS_MIN,LINFOCITOS_MAX,LINFOCITOS_DIFF,NEUTROPHILES_MEDIAN,NEUTROPHILES_MEAN,NEUTROPHILES_MIN,NEUTROPHILES_MAX,NEUTROPHILES_DIFF,P02_ARTERIAL_MEDIAN,P02_ARTERIAL_MEAN,P02_ARTERIAL_MIN,P02_ARTERIAL_MAX,P02_ARTERIAL_DIFF,P02_VENOUS_MEDIAN,P02_VENOUS_MEAN,P02_VENOUS_MIN,P02_VENOUS_MAX,P02_VENOUS_DIFF,PC02_ARTERIAL_MEDIAN,PC02_ARTERIAL_MEAN,PC02_ARTERIAL_MIN,PC02_ARTERIAL_MAX,PC02_ARTERIAL_DIFF,PC02_VENOUS_MEDIAN,PC02_VENOUS_MEAN,PC02_VENOUS_MIN,PC02_VENOUS_MAX,PC02_VENOUS_DIFF,PCR_MEDIAN,PCR_MEAN,PCR_MIN,PCR_MAX,PCR_DIFF,PH_ARTERIAL_MEDIAN,PH_ARTERIAL_MEAN,PH_ARTERIAL_MIN,PH_ARTERIAL_MAX,PH_ARTERIAL_DIFF,PH_VENOUS_MEDIAN,PH_VENOUS_MEAN,PH_VENOUS_MIN,PH_VENOUS_MAX,PH_VENOUS_DIFF,PLATELETS_MEDIAN,PLATELETS_MEAN,PLATELETS_MIN,PLATELETS_MAX,PLATELETS_DIFF,POTASSIUM_MEDIAN,POTASSIUM_MEAN,POTASSIUM_MIN,POTASSIUM_MAX,POTASSIUM_DIFF,SAT02_ARTERIAL_MEDIAN,SAT02_ARTERIAL_MEAN,SAT02_ARTERIAL_MIN,SAT02_ARTERIAL_MAX,SAT02_ARTERIAL_DIFF,SAT02_VENOUS_MEDIAN,SAT02_VENOUS_MEAN,SAT02_VENOUS_MIN,SAT02_VENOUS_MAX,SAT02_VENOUS_DIFF,SODIUM_MEDIAN,SODIUM_MEAN,SODIUM_MIN,SODIUM_MAX,SODIUM_DIFF,TGO_MEDIAN,TGO_MEAN,TGO_MIN,TGO_MAX,TGO_DIFF,TGP_MEDIAN,TGP_MEAN,TGP_MIN,TGP_MAX,TGP_DIFF,TTPA_MEDIAN,TTPA_MEAN,TTPA_MIN,TTPA_MAX,TTPA_DIFF,UREA_MEDIAN,UREA_MEAN,UREA_MIN,UREA_MAX,UREA_DIFF,DIMER_MEDIAN,DIMER_MEAN,DIMER_MIN,DIMER_MAX,DIMER_DIFF,BLOODPRESSURE_DIASTOLIC_MEAN,BLOODPRESSURE_SISTOLIC_MEAN,HEART_RATE_MEAN,RESPIRATORY_RATE_MEAN,TEMPERATURE_MEAN,OXYGEN_SATURATION_MEAN,BLOODPRESSURE_DIASTOLIC_MEDIAN,BLOODPRESSURE_SISTOLIC_MEDIAN,HEART_RATE_MEDIAN,RESPIRATORY_RATE_MEDIAN,TEMPERATURE_MEDIAN,OXYGEN_SATURATION_MEDIAN,BLOODPRESSURE_DIASTOLIC_MIN,BLOODPRESSURE_SISTOLIC_MIN,HEART_RATE_MIN,RESPIRATORY_RATE_MIN,TEMPERATURE_MIN,OXYGEN_SATURATION_MIN,BLOODPRESSURE_DIASTOLIC_MAX,BLOODPRESSURE_SISTOLIC_MAX,HEART_RATE_MAX,RESPIRATORY_RATE_MAX,TEMPERATURE_MAX,OXYGEN_SATURATION_MAX,BLOODPRESSURE_DIASTOLIC_DIFF,BLOODPRESSURE_SISTOLIC_DIFF,HEART_RATE_DIFF,RESPIRATORY_RATE_DIFF,TEMPERATURE_DIFF,OXYGEN_SATURATION_DIFF,BLOODPRESSURE_DIASTOLIC_DIFF_REL,BLOODPRESSURE_SISTOLIC_DIFF_REL,HEART_RATE_DIFF_REL,RESPIRATORY_RATE_DIFF_REL,TEMPERATURE_DIFF_REL,OXYGEN_SATURATION_DIFF_REL,WINDOW,ICU
0,0,1,60th,0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

-----------------
Retornar ao [Sumário](#sumario)

<a name="nan"></a>
# Dados faltantes (NaN)
Para saber mais detalhes sobre a estratégia de eliminação dos dados faltantes (NaN) foram tratados, veja o documento:<br>[0.0_understanding_the_data.md](0.0_understanding_the_data.md), no tópico: **Dados faltantes (NaN)**.

## Existem dados faltantes no DataFrame?
Essa pergunta é importante uma vez que muitos dos modelos de predição não conseguem trabalhar com dados NaN.<br>
A linha conta todas as ocorrencias de 'valores' ```NaN``` no **df**, e vemos que a quantidade é de **223863** NaN. 

In [4]:
print(f'Existem no DataFrame df {df.isna().sum().sum()} "valores" NaN')

Existem no DataFrame df 223863 "valores" NaN


Uma vez que sabemos que existem valores faltantes (NaN) vamos começar a trata-los.

## Limpando os NaN com a função 'neighborhood_missing_data'
Usando a função ```neighborhood_missing_data``` para eliminar os NaN.<br>
**```df_1_without_nan : pd.DataFrame```** é o DataFrame após tirar os NaN com a função ```neighborhood_missing_data```.<br>
Após isso, uma contagem é ralizada para saber se ainda existem dados faltantes.

In [5]:
# Usando a função neighborhood_missing_data para eliminar os NaN
df_1_without_nan = neighborhood_missing_data(df, 'PATIENT_VISIT_IDENTIFIER')

# Avaliando quantos dados ainda sobraram como NaN
print(f'O total de NaN existentes no DataFrame df_1_without_nan é:{df_1_without_nan.isna().sum().sum()}')

O total de NaN existentes no DataFrame df_1_without_nan é:2025


É possível observar uma considerável redução na quantidade de ```NaN```, que passou de: **223863** para: **2025**, mesmo assim, é necessário entender e eliminar esses dados remanescentes.<br>
A função ```print_nan_count_by_feature``` imprime todas as colunas, e mostra a quantidade de dados ```NaN``` das de cada uma.<br>
É possível observar que temos algumas colunas com 5 e outras com 10 NaN. Como a a janela possui 5 perídos ```(0-2, 2-4, 4-6, 6-12 e mais que 12)```, então temos 1 paciente para aquelas que apresentam 5 valores faltantes e 2 pacientes para aquelas que apresentam 10 valores faltantes.

In [6]:
print_nan_count_by_feature(df_1_without_nan)

Count - Feature Name
--------------------
0000  - PATIENT_VISIT_IDENTIFIER
0000  - AGE_ABOVE65
0000  - AGE_PERCENTIL
0000  - GENDER
0005  - DISEASE GROUPING 1
0005  - DISEASE GROUPING 2
0005  - DISEASE GROUPING 3
0005  - DISEASE GROUPING 4
0005  - DISEASE GROUPING 5
0005  - DISEASE GROUPING 6
0005  - HTN
0005  - IMMUNOCOMPROMISED
0005  - OTHER
0010  - ALBUMIN_MEDIAN
0010  - ALBUMIN_MEAN
0010  - ALBUMIN_MIN
0010  - ALBUMIN_MAX
0010  - ALBUMIN_DIFF
0010  - BE_ARTERIAL_MEDIAN
0010  - BE_ARTERIAL_MEAN
0010  - BE_ARTERIAL_MIN
0010  - BE_ARTERIAL_MAX
0010  - BE_ARTERIAL_DIFF
0010  - BE_VENOUS_MEDIAN
0010  - BE_VENOUS_MEAN
0010  - BE_VENOUS_MIN
0010  - BE_VENOUS_MAX
0010  - BE_VENOUS_DIFF
0010  - BIC_ARTERIAL_MEDIAN
0010  - BIC_ARTERIAL_MEAN
0010  - BIC_ARTERIAL_MIN
0010  - BIC_ARTERIAL_MAX
0010  - BIC_ARTERIAL_DIFF
0010  - BIC_VENOUS_MEDIAN
0010  - BIC_VENOUS_MEAN
0010  - BIC_VENOUS_MIN
0010  - BIC_VENOUS_MAX
0010  - BIC_VENOUS_DIFF
0010  - BILLIRUBIN_MEDIAN
0010  - BILLIRUBIN_MEAN
0010  - B

Foi escolhido aleatóriamente uma frature que tem 10 valores NaN, nesse caso a ```UREA_MEDIAN```, e foi feita uma query para selecionar os dados NaN dessa coluna.<br>
É possível observar que existem 2 VISITAS que apresentam NaN, a ID **199** e a ID **287**, isso afirma a hipótese de ser multiplos das 5 janelas.
![alt text](../img/NaN_2_VisitId.png "Timeline_Example_Best")

In [7]:
df_1_without_nan.query('UREA_MEDIAN.isnull()', engine='python')

,PATIENT_VISIT_IDENTIFIER,AGE_ABOVE65,AGE_PERCENTIL,GENDER,DISEASE GROUPING 1,DISEASE GROUPING 2,DISEASE GROUPING 3,DISEASE GROUPING 4,DISEASE GROUPING 5,DISEASE GROUPING 6,HTN,IMMUNOCOMPROMISED,OTHER,ALBUMIN_MEDIAN,ALBUMIN_MEAN,ALBUMIN_MIN,ALBUMIN_MAX,ALBUMIN_DIFF,BE_ARTERIAL_MEDIAN,BE_ARTERIAL_MEAN,BE_ARTERIAL_MIN,BE_ARTERIAL_MAX,BE_ARTERIAL_DIFF,BE_VENOUS_MEDIAN,BE_VENOUS_MEAN,BE_VENOUS_MIN,BE_VENOUS_MAX,BE_VENOUS_DIFF,BIC_ARTERIAL_MEDIAN,BIC_ARTERIAL_MEAN,BIC_ARTERIAL_MIN,BIC_ARTERIAL_MAX,BIC_ARTERIAL_DIFF,BIC_VENOUS_MEDIAN,BIC_VENOUS_MEAN,BIC_VENOUS_MIN,BIC_VENOUS_MAX,BIC_VENOUS_DIFF,BILLIRUBIN_MEDIAN,BILLIRUBIN_MEAN,BILLIRUBIN_MIN,BILLIRUBIN_MAX,BILLIRUBIN_DIFF,BLAST_MEDIAN,BLAST_MEAN,BLAST_MIN,BLAST_MAX,BLAST_DIFF,CALCIUM_MEDIAN,CALCIUM_MEAN,CALCIUM_MIN,CALCIUM_MAX,CALCIUM_DIFF,CREATININ_MEDIAN,CREATININ_MEAN,CREATININ_MIN,CREATININ_MAX,CREATININ_DIFF,FFA_MEDIAN,FFA_MEAN,FFA_MIN,FFA_MAX,FFA_DIFF,GGT_MEDIAN,GGT_MEAN,GGT_MIN,GGT_MAX,GGT_DIFF,GLUCOSE_MEDIAN,GLUCOSE_MEAN,GLUCOSE_MIN,GLUCOSE_MAX,GLUCOSE_DIFF,HEMATOCRITE_MEDIAN,HEMATOCRITE_MEAN,HEMATOCRITE_MIN,HEMATOCRITE_MAX,HEMATOCRITE_DIFF,HEMOGLOBIN_MEDIAN,HEMOGLOBIN_MEAN,HEMOGLOBIN_MIN,HEMOGLOBIN_MAX,HEMOGLOBIN_DIFF,INR_MEDIAN,INR_MEAN,INR_MIN,INR_MAX,INR_DIFF,LACTATE_MEDIAN,LACTATE_MEAN,LACTATE_MIN,LACTATE_MAX,LACTATE_DIFF,LEUKOCYTES_MEDIAN,LEUKOCYTES_MEAN,LEUKOCYTES_MIN,LEUKOCYTES_MAX,LEUKOCYTES_DIFF,LINFOCITOS_MEDIAN,LINFOCITOS_MEAN,LINFOCITOS_MIN,LINFOCITOS_MAX,LINFOCITOS_DIFF,NEUTROPHILES_MEDIAN,NEUTROPHILES_MEAN,NEUTROPHILES_MIN,NEUTROPHILES_MAX,NEUTROPHILES_DIFF,P02_ARTERIAL_MEDIAN,P02_ARTERIAL_MEAN,P02_ARTERIAL_MIN,P02_ARTERIAL_MAX,P02_ARTERIAL_DIFF,P02_VENOUS_MEDIAN,P02_VENOUS_MEAN,P02_VENOUS_MIN,P02_VENOUS_MAX,P02_VENOUS_DIFF,PC02_ARTERIAL_MEDIAN,PC02_ARTERIAL_MEAN,PC02_ARTERIAL_MIN,PC02_ARTERIAL_MAX,PC02_ARTERIAL_DIFF,PC02_VENOUS_MEDIAN,PC02_VENOUS_MEAN,PC02_VENOUS_MIN,PC02_VENOUS_MAX,PC02_VENOUS_DIFF,PCR_MEDIAN,PCR_MEAN,PCR_MIN,PCR_MAX,PCR_DIFF,PH_ARTERIAL_MEDIAN,PH_ARTERIAL_MEAN,PH_ARTERIAL_MIN,PH_ARTERIAL_MAX,PH_ARTERIAL_DIFF,PH_VENOUS_MEDIAN,PH_VENOUS_MEAN,PH_VENOUS_MIN,PH_VENOUS_MAX,PH_VENOUS_DIFF,PLATELETS_MEDIAN,PLATELETS_MEAN,PLATELETS_MIN,PLATELETS_MAX,PLATELETS_DIFF,POTASSIUM_MEDIAN,POTASSIUM_MEAN,POTASSIUM_MIN,POTASSIUM_MAX,POTASSIUM_DIFF,SAT02_ARTERIAL_MEDIAN,SAT02_ARTERIAL_MEAN,SAT02_ARTERIAL_MIN,SAT02_ARTERIAL_MAX,SAT02_ARTERIAL_DIFF,SAT02_VENOUS_MEDIAN,SAT02_VENOUS_MEAN,SAT02_VENOUS_MIN,SAT02_VENOUS_MAX,SAT02_VENOUS_DIFF,SODIUM_MEDIAN,SODIUM_MEAN,SODIUM_MIN,SODIUM_MAX,SODIUM_DIFF,TGO_MEDIAN,TGO_MEAN,TGO_MIN,TGO_MAX,TGO_DIFF,TGP_MEDIAN,TGP_MEAN,TGP_MIN,TGP_MAX,TGP_DIFF,TTPA_MEDIAN,TTPA_MEAN,TTPA_MIN,TTPA_MAX,TTPA_DIFF,UREA_MEDIAN,UREA_MEAN,UREA_MIN,UREA_MAX,UREA_DIFF,DIMER_MEDIAN,DIMER_MEAN,DIMER_MIN,DIMER_MAX,DIMER_DIFF,BLOODPRESSURE_DIASTOLIC_MEAN,BLOODPRESSURE_SISTOLIC_MEAN,HEART_RATE_MEAN,RESPIRATORY_RATE_MEAN,TEMPERATURE_MEAN,OXYGEN_SATURATION_MEAN,BLOODPRESSURE_DIASTOLIC_MEDIAN,BLOODPRESSURE_SISTOLIC_MEDIAN,HEART_RATE_MEDIAN,RESPIRATORY_RATE_MEDIAN,TEMPERATURE_MEDIAN,OXYGEN_SATURATION_MEDIAN,BLOODPRESSURE_DIASTOLIC_MIN,BLOODPRESSURE_SISTOLIC_MIN,HEART_RATE_MIN,RESPIRATORY_RATE_MIN,TEMPERATURE_MIN,OXYGEN_SATURATION_MIN,BLOODPRESSURE_DIASTOLIC_MAX,BLOODPRESSURE_SISTOLIC_MAX,HEART_RATE_MAX,RESPIRATORY_RATE_MAX,TEMPERATURE_MAX,OXYGEN_SATURATION_MAX,BLOODPRESSURE_DIASTOLIC_DIFF,BLOODPRESSURE_SISTOLIC_DIFF,HEART_RATE_DIFF,RESPIRATORY_RATE_DIFF,TEMPERATURE_DIFF,OXYGEN_SATURATION_DIFF,BLOODPRESSURE_DIASTOLIC_DIFF_REL,BLOODPRESSURE_SISTOLIC_DIFF_REL,HEART_RATE_DIFF_REL,RESPIRATORY_RATE_DIFF_REL,TEMPERATURE_DIFF_REL,OXYGEN_SATURATION_DIFF_REL,WINDOW,ICU
995,199,0,10th,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

**```df_2_without_nan : pd.DataFrame```** é a ```variável df_1_without_nan``` após o drop dos index 199 e 287, que continham valores NaN

In [8]:
df_2_without_nan = df_1_without_nan.drop(df_1_without_nan.query('UREA_MEDIAN.isnull()', engine='python').index)
print(f'A quantidade de valores NaN após a remocção dos dois IDs de visita é: {df_2_without_nan.isna().sum().sum()}')

A quantidade de valores NaN após a remocção dos dois IDs de visita é: 0


-----------------
Retornar ao [Sumário](#sumario)

<a name="repet"></a>
# Valores repetidos
Essa etapa procura *features* ou colunas com um único valor repetido em todas as linhas ou observações.<br>
Uma feature com um único valor repetido em todas as linhas é desnecessária para o modelo, uma vez que não existe variação, logo, se não serve para o modelo de predição, deve ser removido do DataFrame.<br>
A função ```plot_features_with_same_value_for_all_observations``` busca por colunas nessa condição, e plota o nome das colunas.<br>
Existem 36 resultados relacionados com **exames de sangue**, e note que o total de colunas repetidas é 36. Isso porque o exame de sangue é coletado apenas uma vez por dia, não tendo assim um novo exame para se calcular a **diferença** entre a medição anterior e a atual, ou seja, os resultados relacionados com o exame de sangue com o sulfixo **_DIFF** podem ser removidos do DataFrame sem impactos negativos para a modelagem.

In [9]:
plot_features_with_same_value_for_all_observations(df_2_without_nan)

Nome das Colunas:
--------------------
ALBUMIN_DIFF
BE_ARTERIAL_DIFF
BE_VENOUS_DIFF
BIC_ARTERIAL_DIFF
BIC_VENOUS_DIFF
BILLIRUBIN_DIFF
BLAST_DIFF
CALCIUM_DIFF
CREATININ_DIFF
FFA_DIFF
GGT_DIFF
GLUCOSE_DIFF
HEMATOCRITE_DIFF
HEMOGLOBIN_DIFF
INR_DIFF
LACTATE_DIFF
LEUKOCYTES_DIFF
LINFOCITOS_DIFF
NEUTROPHILES_DIFF
P02_ARTERIAL_DIFF
P02_VENOUS_DIFF
PC02_ARTERIAL_DIFF
PC02_VENOUS_DIFF
PCR_DIFF
PH_ARTERIAL_DIFF
PH_VENOUS_DIFF
PLATELETS_DIFF
POTASSIUM_DIFF
SAT02_ARTERIAL_DIFF
SAT02_VENOUS_DIFF
SODIUM_DIFF
TGO_DIFF
TGP_DIFF
TTPA_DIFF
UREA_DIFF
DIMER_DIFF
--------------------
Total: 36


A função ```drop_features_with_same_value_for_all_observations``` remove do DataFrame essas colunas apontadas pela célula anterior.<br>
**```df_3_without_same_value_col : pd.DataFrame```** é o DataFrame após a remoção das colunas com valores repetidos.

In [10]:
df_3_without_same_value_col = drop_features_with_same_value_for_all_observations(df_2_without_nan, False)

Total of dropped columns: 36


Confirmação de que todas as colunas com valores repetidos foram excluidas com sucesso.

In [11]:
plot_features_with_same_value_for_all_observations(df_3_without_same_value_col)

Nome das Colunas:
--------------------
--------------------
Total: 0


-----------------
Retornar ao [Sumário](#sumario)

<a name="dup_feat"></a>
# Features duplicadas
Essa etapa procura por *features* ou colunas que tem os valores idênticos a uma ou mais colunas.<br>
Features duplicadas são desnecessárias para o modelo de regressão, uma vez que não trazem nenhuma informação nova para o modelo, e devem ser removidas do DataFrame.<br>
A função ```plot_duplicated_features``` vai mostrar quais *features* são duplicadas. quando existem duas ou mais *features* identicas, a função deixa a primeira e retorna todas as demais, indicando quais devem ser excluidas.

In [12]:
plot_duplicated_features(df_3_without_same_value_col)

Nome das Colunas:
--------------------
ALBUMIN_MEAN
ALBUMIN_MIN
ALBUMIN_MAX
BE_ARTERIAL_MEAN
BE_ARTERIAL_MIN
BE_ARTERIAL_MAX
BE_VENOUS_MEAN
BE_VENOUS_MIN
BE_VENOUS_MAX
BIC_ARTERIAL_MEAN
BIC_ARTERIAL_MIN
BIC_ARTERIAL_MAX
BIC_VENOUS_MEAN
BIC_VENOUS_MIN
BIC_VENOUS_MAX
BILLIRUBIN_MEAN
BILLIRUBIN_MIN
BILLIRUBIN_MAX
BLAST_MEAN
BLAST_MIN
BLAST_MAX
CALCIUM_MEAN
CALCIUM_MIN
CALCIUM_MAX
CREATININ_MEAN
CREATININ_MIN
CREATININ_MAX
FFA_MEAN
FFA_MIN
FFA_MAX
GGT_MEAN
GGT_MIN
GGT_MAX
GLUCOSE_MEAN
GLUCOSE_MIN
GLUCOSE_MAX
HEMATOCRITE_MEAN
HEMATOCRITE_MIN
HEMATOCRITE_MAX
HEMOGLOBIN_MEAN
HEMOGLOBIN_MIN
HEMOGLOBIN_MAX
INR_MEAN
INR_MIN
INR_MAX
LACTATE_MEAN
LACTATE_MIN
LACTATE_MAX
LEUKOCYTES_MEAN
LEUKOCYTES_MIN
LEUKOCYTES_MAX
LINFOCITOS_MEAN
LINFOCITOS_MIN
LINFOCITOS_MAX
NEUTROPHILES_MEAN
NEUTROPHILES_MIN
NEUTROPHILES_MAX
P02_ARTERIAL_MEAN
P02_ARTERIAL_MIN
P02_ARTERIAL_MAX
P02_VENOUS_MEAN
P02_VENOUS_MIN
P02_VENOUS_MAX
PC02_ARTERIAL_MEAN
PC02_ARTERIAL_MIN
PC02_ARTERIAL_MAX
PC02_VENOUS_MEAN
PC02_VENOUS_MIN
PC0

Existem então **108** *features* duplicadas nesse DataFrame. Se é possivel observar que **108** é um multiplo de **36**, ou seja, os dados de sangue, por possuírem um único valor por visita de paciente (uma única medição de sangue por paciente), isso faz com que as características como: ```MIN```, ```MAX```, ```MEAN``` e ```MEDIAN``` sejam iguais.

In [13]:
single_value_array = [5]
print(f'Média: {np.mean(single_value_array)}\nMediana: {np.median(single_value_array)}\nMínimo: {np.min(single_value_array)}\nMáximo: {np.max(single_value_array)}')

Média: 5.0
Mediana: 5.0
Mínimo: 5
Máximo: 5


É necessário então remover as colunas duplicadas, e para tal, foi criada a função ```drop_duplicated_features```, que elimina do DataFrame as features duplicadas.<br>
**```df_4_without_duplicated_features : pd.DataFrame```** é o DataFrame contendo os valores do ```df_3_without_same_value_col``` com as features duplicadas removidas.

In [14]:
df_4_without_duplicated_features = drop_duplicated_features(df_3_without_same_value_col, False)

Total dropped columns: 108


Confirmando que essas colunas foram removidas:

In [15]:
plot_duplicated_features(df_4_without_duplicated_features)

Nome das Colunas:
--------------------
--------------------
Total: 0


-----------------
Retornar ao [Sumário](#sumario)

<a name="rename"></a>
# Nome features exame de sangue

## Caso dos nomes
Compunham o *raw data* 36 exames de sangue, como: ```[ALBUMIN, BE_ARTERIAL, BE_VENOUS, BIC_ARTERIAL]```.<br>
Cada exame tinham 5 features associadas, sendo ```[MEDIAN, MEAN, MIN, MAX e DIFF]```.<br>
Como o exame de sangue é feito somente uma vez, o valor da ```DIFF``` é irrelevante, bem como os valores de ```MAX```, ```MIN``` e ```MEAN``` que são iguais ao valor de ```MEDIAN```.<br>
Após todo o processamento que fizemos até aqui, a sobraram no DataFrame 36 features relacionadas com o exame de sangue, só que com o sulfixo ```_MEDIAN```. O problema é que é inapropriado apontar a Mediana de um conjunto contendo um único valor. Isso poderia confundir uma pessoa que está por avaliar o modelo, por esse motivo, os sulfixos ```_MEDIAN``` serão removidos dos nomes das features relacionadas com o exame de sangue.

In [17]:
df_5_renamed = rename_portion_of_columns(df_4_without_duplicated_features, 13, (13+36), '_MEDIAN', '')
df_5_renamed.columns[13: (13+36)]

Index(['ALBUMIN', 'BE_ARTERIAL', 'BE_VENOUS', 'BIC_ARTERIAL', 'BIC_VENOUS',
       'BILLIRUBIN', 'BLAST', 'CALCIUM', 'CREATININ', 'FFA', 'GGT', 'GLUCOSE',
       'HEMATOCRITE', 'HEMOGLOBIN', 'INR', 'LACTATE', 'LEUKOCYTES',
       'LINFOCITOS', 'NEUTROPHILES', 'P02_ARTERIAL', 'P02_VENOUS',
       'PC02_ARTERIAL', 'PC02_VENOUS', 'PCR', 'PH_ARTERIAL', 'PH_VENOUS',
       'PLATELETS', 'POTASSIUM', 'SAT02_ARTERIAL', 'SAT02_VENOUS', 'SODIUM',
       'TGO', 'TGP', 'TTPA', 'UREA', 'DIMER'],
      dtype='object')

-----------------
Retornar ao [Sumário](#sumario)

<a name="win_select"></a>
# Window Selection

-----------------
Retornar ao [Sumário](#sumario)

# Conclusão

-----------------
Retornar ao [Sumário](#sumario)